In [50]:
import os
os.chdir("C:\\Users\\Thais\\Documents\\Python\\bcb-sentiment-analysis")

import datetime 

import pandas as pd
import numpy as np
from pandas.tseries.offsets import BDay
import scipy as sc
import statsmodels.api as sm
import pickle
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

DAILY_DATA = ".\\data\\financial data\\daily.xlsx"
MINUTES_DATA = ".\\data\\minutes\\copom_dates.xlsx"
MINUTES_SCORE = ".\\data\\minutes\\minutes_scores_uncased_20230423.pkl"
TARGET_INFLATION_DATA = ".\\data\\financial data\\historico_meta_inflacao.xlsx"
INDEPENDENT_VARIABLES = ".\\data\\output\\independent.xlsx"
INDEPENDENT_VARIABLES_ROBUST = ".\\data\\output\\independent_robust.xlsx"
INDEPENDENT_VARIABLES_ALT = ".\\data\\output\\independent_no_ffill.xlsx"
INDEPENDENT_VARIABLES_ROBUST_ALT = ".\\data\\output\\independent_robust_no_ffill.xlsx"

last_date = datetime.datetime(2023, 4, 11)

# 1 - Processing Data

### 1.1 Daily Historical Volatility

In [11]:
tenors_headers = ["21D","42D","63D","84D","105D","126D","147D","168D","189D","210D","231D","252D","273D","294D","504D","756D","1260D","1512D","1764D","2016D","2268D","2520D","2772D"]
daily_vol = pd.read_excel(DAILY_DATA, sheet_name='vol', skiprows=5).set_index('Dates')
daily_vol.columns = tenors_headers

### 1.2 - IPCA series

In [20]:
#Diário
ipca_yoy_desv_centro = pd.read_excel(TARGET_INFLATION_DATA, sheet_name='ipca_yoy_ano_corrente').set_index('date')[['desvio_centro_perc']]
ipca_yoy_desv_interv = pd.read_excel(TARGET_INFLATION_DATA, sheet_name='ipca_yoy_ano_corrente').set_index('date')[['desvio_intervalo_perc']]

#Dias que saiu IPCA
ipca_mom_desv_expect = pd.read_excel(TARGET_INFLATION_DATA, sheet_name='ipca_mom_ano_corrente').set_index('date')[['desvio_expectativa_perc']]

#Diário
ipca_hpm_desv_centro = pd.read_excel(TARGET_INFLATION_DATA, sheet_name='ipca_horizonte_PM').set_index('date')[['desvio_centro_perc']]
ipca_hpm_desv_interv = pd.read_excel(TARGET_INFLATION_DATA, sheet_name='ipca_horizonte_PM').set_index('date')[['desvio_intervalo_perc']]


### 1.3 - SELIC Surprise

In [21]:
selic_raw = pd.read_excel(MINUTES_DATA, sheet_name = 'Plan1').set_index('date')

In [22]:
selic_surprise_v = selic_raw[['decision_date','selic_surprise_v']].reset_index().set_index('decision_date').drop(columns = ['date'])
selic_frd = selic_raw[['decision_date','frd_guidance']].reset_index().set_index('decision_date').drop(columns = ['date'])
d_inicio = pd.DataFrame(selic_raw['d_inicio'])
d_fim = pd.DataFrame(selic_raw['d_fim'])

In [23]:
d_ata = pd.read_excel(MINUTES_DATA, sheet_name = 'd_ata').set_index('date')
d_ato = pd.read_excel(MINUTES_DATA, sheet_name = 'd_ato').set_index('date')

### 1.4 Minutes score

In [37]:
minute_score = pd.read_pickle(MINUTES_SCORE).reset_index().set_index('date')[['score']]

### 1.5 - CDS

In [24]:
col_headers = ["CO1 Comdty","CL1 Comdty","CRB CMDT Index","VIX Index","BRAZIL CDS USD SR 5Y D14 Corp"]
cds = pd.read_excel(DAILY_DATA, sheet_name='other_controls', skiprows=5).set_index('Dates')
cds.columns = col_headers
cds_s = cds[["BRAZIL CDS USD SR 5Y D14 Corp"]]


# 2 - Joining all in one dataframe

### Independent Variables

In [7]:
start_date = datetime.datetime(2001, 10, 12) #data que todas as séries tem dado
end_date = datetime.datetime(2022, 12, 13) #última ata

In [20]:
#Regenerate data resampled
ls_df = [cds_s, minute_score, selic_frd, selic_surprise_v, ipca_mom_desv_expect,
        ipca_yoy_desv_interv, ipca_yoy_desv_centro, ipca_hpm_desv_interv, ipca_hpm_desv_centro, d_ata, d_ato,
        d_inicio, d_fim]

new_ls_df = []
for df in ls_df:
    df.index.name = 'date'
    temp = df.resample('D').ffill()
    
    df_filtered = temp[(temp.index >= start_date) & (temp.index <= end_date)]

    new_ls_df.append(df_filtered)

df_independent = pd.concat(new_ls_df, axis = 1)

In [42]:
column_headers = ['CDS', 'minutes_score', 'frd_guidance','selic_surprise_v', 'desvio_expectativa_perc',
                           'desvio_intervalo_perc','desvio_centro_perc', 'desvio_intervalo_hpm_perc', 
                           'desvio_centro_hpm_perc', 'd_ata', 'd_ato', 'd_inicio', 'd_fim']

df_independent = pd.read_excel(INDEPENDENT_VARIABLES_ALT).set_index('date')
df_independent.columns = column_headers

### Dependent Variable

In [12]:
df_vol = daily_vol.resample('D').ffill() 
df_dependent = df_vol[(df_vol.index >= start_date) & (df_vol.index <= end_date)]

# 3 - Regressions

## 3.1 - Sem controles

### 3.1.0 - vol_di = NLP

In [43]:
ls_df = []

for tenor in tenors_headers:

    # define column of tenors
    lst_tenor = [tenor]
    rep_lst_tenors = np.repeat(lst_tenor, [1+1], axis=0)

    # define your dependent variable
    y = df_dependent[tenor]

    # define your independent variables
    x1 = df_independent['minutes_score']

    # create a matrix of your independent variables
    X = sm.add_constant(pd.concat([x1], axis=1))

    # create the OLS model and fit it to your data
    model = sm.OLS(y, X, missing='drop').fit()

    # obtain a summary table in a structured format
    summary_table = model.summary2()

    # extract the p-values ans SD columns from the table
    p_values = summary_table.tables[1]['P>|t|']
    sd = summary_table.tables[1]['Std.Err.']
    beta = summary_table.tables[1]['Coef.']

    # create a pandas dataframe to store the p-values
    p_values_df = pd.DataFrame({'variable': p_values.index, 'tenor': rep_lst_tenors, 'beta':beta.values, 
                                'sd':sd.values, 'p-value': p_values.values})
    ls_df.append(p_values_df)

df_values_310 = pd.concat(ls_df)

### 3.1.1 - vol_di = NLP + D_decisão + D_ata + Selic Surprise value

In [44]:
ls_df = []

for tenor in tenors_headers:

    # define column of tenors
    lst_tenor = [tenor]
    rep_lst_tenors = np.repeat(lst_tenor, [4+1], axis=0)

    # define your dependent variable
    y = df_dependent[tenor]

    # define your independent variables
    x1 = df_independent['minutes_score']
    x2 = df_independent['selic_surprise_v']
    x3 = df_independent['d_ato']
    x4 = df_independent['d_ata']

    # create a matrix of your independent variables
    X = sm.add_constant(pd.concat([x1, x2, x3, x4], axis=1))

    # create the OLS model and fit it to your data
    model = sm.OLS(y, X, missing='drop').fit()

    # obtain a summary table in a structured format
    summary_table = model.summary2()

    # extract the p-values ans SD columns from the table
    p_values = summary_table.tables[1]['P>|t|']
    sd = summary_table.tables[1]['Std.Err.']
    beta = summary_table.tables[1]['Coef.']

    # create a pandas dataframe to store the p-values
    p_values_df = pd.DataFrame({'variable': p_values.index, 'tenor': rep_lst_tenors, 'beta':beta.values, 
                                'sd':sd.values, 'p-value': p_values.values})
    ls_df.append(p_values_df)

df_values_311 = pd.concat(ls_df)

### 3.1.2 - vol_di = NLP + D_decisão + Selic Surprise value

In [45]:
ls_df = []

for tenor in tenors_headers:

    # define column of tenors
    lst_tenor = [tenor]
    rep_lst_tenors = np.repeat(lst_tenor, [3+1], axis=0)

    # define your dependent variable
    y = df_dependent[tenor]

    # define your independent variables
    x1 = df_independent['minutes_score']
    x2 = df_independent['selic_surprise_v']
    x3 = df_independent['d_ato']

    # create a matrix of your independent variables
    X = sm.add_constant(pd.concat([x1, x2, x3], axis=1))

    # create the OLS model and fit it to your data
    model = sm.OLS(y, X, missing='drop').fit()

    # obtain a summary table in a structured format
    summary_table = model.summary2()

    # extract the p-values ans SD columns from the table
    p_values = summary_table.tables[1]['P>|t|']
    sd = summary_table.tables[1]['Std.Err.']
    beta = summary_table.tables[1]['Coef.']

    # create a pandas dataframe to store the p-values
    p_values_df = pd.DataFrame({'variable': p_values.index, 'tenor': rep_lst_tenors, 'beta':beta.values, 
                                'sd':sd.values, 'p-value': p_values.values})
    ls_df.append(p_values_df)

df_values_312 = pd.concat(ls_df)

### 3.1.3 - vol_di = NLP + D_ata + Selic Surprise value

In [46]:
ls_df = []

for tenor in tenors_headers:

    # define column of tenors
    lst_tenor = [tenor]
    rep_lst_tenors = np.repeat(lst_tenor, [3+1], axis=0)

    # define your dependent variable
    y = df_dependent[tenor]

    # define your independent variables
    x1 = df_independent['minutes_score']
    x2 = df_independent['selic_surprise_v']
    x3 = df_independent['d_ata']

    # create a matrix of your independent variables
    X = sm.add_constant(pd.concat([x1, x2, x3], axis=1))

    # create the OLS model and fit it to your data
    model = sm.OLS(y, X, missing='drop').fit()

    # obtain a summary table in a structured format
    summary_table = model.summary2()

    # extract the p-values ans SD columns from the table
    p_values = summary_table.tables[1]['P>|t|']
    sd = summary_table.tables[1]['Std.Err.']
    beta = summary_table.tables[1]['Coef.']

    # create a pandas dataframe to store the p-values
    p_values_df = pd.DataFrame({'variable': p_values.index, 'tenor': rep_lst_tenors, 'beta':beta.values, 
                                'sd':sd.values, 'p-value': p_values.values})
    ls_df.append(p_values_df)

df_values_313 = pd.concat(ls_df)

## 3.2 - Com controles

### 3.2.1 - vol_di = NLP + D_ato + D_ata + ... + Selic Surprise valores

In [47]:
ls_df = []

for tenor in tenors_headers:

    # define column of tenors
    lst_tenor = [tenor]
    rep_lst_tenors = np.repeat(lst_tenor, [13+1], axis=0)

    # define your dependent variable
    y = df_dependent[tenor]

    # define your independent variables
    x1 = df_independent['minutes_score']
    x2 = df_independent['selic_surprise_v']
    x3 = df_independent['d_ato']
    x4 = df_independent['d_ata']
    x5 = df_independent['frd_guidance']
    x6 = df_independent['desvio_expectativa_perc']
    x7 = df_independent['desvio_intervalo_perc']
    x8 = df_independent['desvio_centro_perc']
    x9 = df_independent['desvio_intervalo_hpm_perc']
    x10 = df_independent['desvio_centro_hpm_perc']
    x11 = df_independent['CDS']
    x12 = df_independent['d_inicio']
    x13 = df_independent['d_fim']

    # create a matrix of your independent variables
    X = sm.add_constant(pd.concat([x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13], axis=1))

    # create the OLS model and fit it to your data
    model = sm.OLS(y, X, missing='drop').fit()

    # obtain a summary table in a structured format
    summary_table = model.summary2()

    # extract the p-values ans SD columns from the table
    p_values = summary_table.tables[1]['P>|t|']
    sd = summary_table.tables[1]['Std.Err.']
    beta = summary_table.tables[1]['Coef.']

    # create a pandas dataframe to store the p-values
    p_values_df = pd.DataFrame({'variable': p_values.index, 'tenor': rep_lst_tenors, 'beta':beta.values, 
                                'sd':sd.values, 'p-value': p_values.values})

    ls_df.append(p_values_df)

df_values_321 = pd.concat(ls_df)

# Save to excel

In [48]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter("modelos_20230608_alt.xlsx", engine="xlsxwriter")

# Write each dataframe to a different worksheet.
df_values_310.to_excel(writer, sheet_name="df_values_310")
df_values_311.to_excel(writer, sheet_name="df_values_311")
df_values_312.to_excel(writer, sheet_name="df_values_312")
df_values_313.to_excel(writer, sheet_name="df_values_313")
df_values_321.to_excel(writer, sheet_name="df_values_321")

# Close the Pandas Excel writer and output the Excel file.
writer.close()

# Robustness Analysis

In [4]:
MINUTES_SCORE_UNCASED = ".\\data\\minutes\\minutes_scores_uncased.pkl"
minute_score_uncased = pd.read_pickle(MINUTES_SCORE_UNCASED).reset_index().set_index('date')[['score']].resample('D').ffill()
minute_score_uncased = minute_score_uncased[(minute_score_uncased.index >= start_date) & (minute_score_uncased.index <= end_date)]

In [27]:
#Regenerate data resampled
ls_df = [cds_s, minute_score_uncased, selic_frd, selic_surprise_v, ipca_mom_desv_expect,
        ipca_yoy_desv_interv, ipca_yoy_desv_centro, ipca_hpm_desv_interv, ipca_hpm_desv_centro, d_ata, d_ato,
        d_inicio, d_fim]

new_ls_df = []
for df in ls_df:
    df.index.name = 'date'
    temp = df.resample('D').ffill()
    
    df_filtered = temp[(temp.index >= start_date) & (temp.index <= end_date)]

    new_ls_df.append(df_filtered)

df_independent = pd.concat(new_ls_df, axis = 1)


In [51]:
column_headers = ['CDS', 'minutes_score', 'frd_guidance','selic_surprise_v', 'desvio_expectativa_perc',
                           'desvio_intervalo_perc','desvio_centro_perc', 'desvio_intervalo_hpm_perc', 
                           'desvio_centro_hpm_perc', 'd_ata', 'd_ato', 'd_inicio', 'd_fim']

df_independent = pd.read_excel(INDEPENDENT_VARIABLES_ROBUST_ALT).set_index('date')
df_independent.columns = column_headers

### 310

In [52]:
ls_df = []

for tenor in tenors_headers:

    # define column of tenors
    lst_tenor = [tenor]
    rep_lst_tenors = np.repeat(lst_tenor, [1+1], axis=0)

    # define your dependent variable
    y = df_dependent[tenor]

    # define your independent variables
    x1 = df_independent['minutes_score']

    # create a matrix of your independent variables
    X = sm.add_constant(pd.concat([x1], axis=1))

    # create the OLS model and fit it to your data
    model = sm.OLS(y, X, missing='drop').fit()

    # obtain a summary table in a structured format
    summary_table = model.summary2()

    # extract the p-values ans SD columns from the table
    p_values = summary_table.tables[1]['P>|t|']
    sd = summary_table.tables[1]['Std.Err.']
    beta = summary_table.tables[1]['Coef.']

    # create a pandas dataframe to store the p-values
    p_values_df = pd.DataFrame({'variable': p_values.index, 'tenor': rep_lst_tenors, 'beta':beta.values, 
                                'sd':sd.values, 'p-value': p_values.values})
    ls_df.append(p_values_df)

df_values_r310 = pd.concat(ls_df)

### 311

In [53]:
ls_df = []

for tenor in tenors_headers:

    # define column of tenors
    lst_tenor = [tenor]
    rep_lst_tenors = np.repeat(lst_tenor, [4+1], axis=0)

    # define your dependent variable
    y = df_dependent[tenor]

    # define your independent variables
    x1 = df_independent['minutes_score']
    x2 = df_independent['selic_surprise_v']
    x3 = df_independent['d_ato']
    x4 = df_independent['d_ata']

    # create a matrix of your independent variables
    X = sm.add_constant(pd.concat([x1, x2, x3, x4], axis=1))

    # create the OLS model and fit it to your data
    model = sm.OLS(y, X, missing='drop').fit()

    # obtain a summary table in a structured format
    summary_table = model.summary2()

    # extract the p-values ans SD columns from the table
    p_values = summary_table.tables[1]['P>|t|']
    sd = summary_table.tables[1]['Std.Err.']
    beta = summary_table.tables[1]['Coef.']

    # create a pandas dataframe to store the p-values
    p_values_df = pd.DataFrame({'variable': p_values.index, 'tenor': rep_lst_tenors, 'beta':beta.values, 
                                'sd':sd.values, 'p-value': p_values.values})
    ls_df.append(p_values_df)

df_values_r311 = pd.concat(ls_df)

### 312

In [54]:
ls_df = []

for tenor in tenors_headers:

    # define column of tenors
    lst_tenor = [tenor]
    rep_lst_tenors = np.repeat(lst_tenor, [3+1], axis=0)

    # define your dependent variable
    y = df_dependent[tenor]

    # define your independent variables
    x1 = df_independent['minutes_score']
    x2 = df_independent['selic_surprise_v']
    x3 = df_independent['d_ato']

    # create a matrix of your independent variables
    X = sm.add_constant(pd.concat([x1, x2, x3], axis=1))

    # create the OLS model and fit it to your data
    model = sm.OLS(y, X, missing='drop').fit()

    # obtain a summary table in a structured format
    summary_table = model.summary2()

    # extract the p-values ans SD columns from the table
    p_values = summary_table.tables[1]['P>|t|']
    sd = summary_table.tables[1]['Std.Err.']
    beta = summary_table.tables[1]['Coef.']

    # create a pandas dataframe to store the p-values
    p_values_df = pd.DataFrame({'variable': p_values.index, 'tenor': rep_lst_tenors, 'beta':beta.values, 
                                'sd':sd.values, 'p-value': p_values.values})
    ls_df.append(p_values_df)

df_values_r312 = pd.concat(ls_df)

### 313

In [55]:
ls_df = []

for tenor in tenors_headers:

    # define column of tenors
    lst_tenor = [tenor]
    rep_lst_tenors = np.repeat(lst_tenor, [3+1], axis=0)

    # define your dependent variable
    y = df_dependent[tenor]

    # define your independent variables
    x1 = df_independent['minutes_score']
    x2 = df_independent['selic_surprise_v']
    x3 = df_independent['d_ata']

    # create a matrix of your independent variables
    X = sm.add_constant(pd.concat([x1, x2, x3], axis=1))

    # create the OLS model and fit it to your data
    model = sm.OLS(y, X, missing='drop').fit()

    # obtain a summary table in a structured format
    summary_table = model.summary2()

    # extract the p-values ans SD columns from the table
    p_values = summary_table.tables[1]['P>|t|']
    sd = summary_table.tables[1]['Std.Err.']
    beta = summary_table.tables[1]['Coef.']

    # create a pandas dataframe to store the p-values
    p_values_df = pd.DataFrame({'variable': p_values.index, 'tenor': rep_lst_tenors, 'beta':beta.values, 
                                'sd':sd.values, 'p-value': p_values.values})
    ls_df.append(p_values_df)

df_values_r313 = pd.concat(ls_df)

### 321

In [56]:
ls_df = []

for tenor in tenors_headers:

    # define column of tenors
    lst_tenor = [tenor]
    rep_lst_tenors = np.repeat(lst_tenor, [13+1], axis=0)

    # define your dependent variable
    y = df_dependent[tenor]

    # define your independent variables
    x1 = df_independent['minutes_score']
    x2 = df_independent['selic_surprise_v']
    x3 = df_independent['d_ato']
    x4 = df_independent['d_ata']
    x5 = df_independent['frd_guidance']
    x6 = df_independent['desvio_expectativa_perc']
    x7 = df_independent['desvio_intervalo_perc']
    x8 = df_independent['desvio_centro_perc']
    x9 = df_independent['desvio_intervalo_hpm_perc']
    x10 = df_independent['desvio_centro_hpm_perc']
    x11 = df_independent['CDS']
    x12 = df_independent['d_inicio']
    x13 = df_independent['d_fim']

    # create a matrix of your independent variables
    X = sm.add_constant(pd.concat([x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13], axis=1))

    # create the OLS model and fit it to your data
    model = sm.OLS(y, X, missing='drop').fit()

    # obtain a summary table in a structured format
    summary_table = model.summary2()

    # extract the p-values ans SD columns from the table
    p_values = summary_table.tables[1]['P>|t|']
    sd = summary_table.tables[1]['Std.Err.']
    beta = summary_table.tables[1]['Coef.']

    # create a pandas dataframe to store the p-values
    p_values_df = pd.DataFrame({'variable': p_values.index, 'tenor': rep_lst_tenors, 'beta':beta.values, 
                                'sd':sd.values, 'p-value': p_values.values})

    ls_df.append(p_values_df)

df_values_r321 = pd.concat(ls_df)

## Save to Excel

In [57]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter("summary_robustness_alt.xlsx", engine="xlsxwriter")

# Write each dataframe to a different worksheet.
df_values_r310.to_excel(writer, sheet_name="df_values_r310")
df_values_r311.to_excel(writer, sheet_name="df_values_r311")
df_values_r312.to_excel(writer, sheet_name="df_values_r312")
df_values_r313.to_excel(writer, sheet_name="df_values_r313")
df_values_r321.to_excel(writer, sheet_name="df_values_r321")

# Close the Pandas Excel writer and output the Excel file.
writer.close()

# Correlation Matrix

In [ ]:
#Fazer entre minutes_score e SELIC + EXPECTATIVA INFLAÇÃO 12m (no dia da decisão) + EXPECTATIVA GDP anual (no dia da decisão) + FX expectativa de final de ano (no dia da decisão)

In [ ]:
# df_independent.columns

In [ ]:
# df_corr = df_independent[['FCI','minutes_score']]

In [ ]:
# plt.matshow(df_corr.corr())
# plt.show()